## Installation

Installing Bio, transformers, genomic-benchmarks, and datasets packages.  The Bio package is from Biopython; transformers package for machine learning (pytorch, tensorflow); genomic-benchmarks and datasets from ML-Bioinfo-CEITEC.


In [ ]:
# already set up on Expanse; toggle for colab

# pip install -qq Bio transformers genomic-benchmarks datasets transformers[torch] pyfaidx

Set path prefix for Expanse

In [1]:
path_prefix = "/expanse/lustre/projects/nji102/sgriesmer/"

Import SNP datasets

In [2]:
import pandas as pd

abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/RARPAT_0_102-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')
abs_diff_plus_log_odds_scores_norm.head(), abs_diff_plus_log_odds_scores_norm.shape


(   BroadDnd41CtcfUniPk151-ran  BroadDnd41Ezh239875UniPk151-ran  \
 0                   -0.177281                         3.297896   
 1                   -0.171037                        -0.342376   
 2                   -0.177402                        -0.587210   
 3                   -0.177683                        -0.649530   
 4                   -0.177163                        -0.598919   
 
    BroadGm12878CtcfUniPk151-ran  BroadGm12878Ezh239875UniPk151-ran  \
 0                     -0.245501                           0.650651   
 1                     -0.260220                           0.278552   
 2                      0.076961                          -0.745354   
 3                     -0.285416                          -0.986173   
 4                     -0.206958                           0.028644   
 
    BroadH1hescChd1a301218aUniPk151-ran  BroadH1hescCtcfUniPk151-ran  \
 0                             5.995141                    -0.249386   
 1                      

Choose test set for run

In [3]:
import xgboost

/cm/shared/apps/spack/0.17.3/cpu/b/opt/spack/linux-rocky8-zen/gcc-8.5.0/anaconda3-2021.05-q4munrgvh7qp4o7r3nzcdkbuph4z7375/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.24.4)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [4]:
from numpy import asarray
from numpy import mean
from numpy import std
from sklearn.datasets import make_classification
from xgboost import XGBClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from matplotlib import pyplot

In [6]:
X=abs_diff_plus_log_odds_scores_norm[0:102]

In [7]:
X

,BroadDnd41CtcfUniPk151-ran,BroadDnd41Ezh239875UniPk151-ran,BroadGm12878CtcfUniPk151-ran,BroadGm12878Ezh239875UniPk151-ran,BroadH1hescChd1a301218aUniPk151-ran,BroadH1hescCtcfUniPk151-ran,BroadH1hescEzh239875UniPk151-ran,BroadH1hescJarid1aab26049UniPk151-ran,BroadH1hescRbbp5a300109aUniPk151-ran,BroadHelas3CtcfUniPk151-ran,...,UwMcf7CtcfUniPk151-ran.1,UwNb4CtcfUniPk151-ran.1,UwNhdfneoCtcfUniPk151-ran.1,UwNhekCtcfUniPk151-ran.1,UwNhlfCtcfUniPk151-ran.1,UwRptecCtcfUniPk151-ran.1,UwSaecCtcfUniPk151-ran.1,UwSknshraCtcfUniPk151-ran.1,UwWerirb1CtcfUniPk151-ran.1,UwWi38CtcfUniPk151-ran.1
0,-0.177281,3.297896,-0.245501,0.650651,5.995141,-0.249386,-0.306055,-0.413578,1.077594,-0.334877,...,-0.298899,-0.287583,-0.148622,-0.234587,-0.290536,-0.640381,-0.067265,-0.376702,-0.296578,-0.428227
1,-0.171037,-0.342376,-0.260220,0.278552,-0.522190,-0.242656,-0.314110,-0.439076,-0.423111,-0.283096,...,-0.416881,-0.295374,-0.593796,-0.209265,-0.034916,-0.381576,-0.552161,-0.366302,-0.277084,-0.488131
2,-0.177402,-0.587210,0.076961,-0.745354,-0.522509,-0.187210,-0.314593,-0.091367,-0.422259,-0.326698,...,-0.409615,0.252402,-0.544256,0.474081,0.388900,0.740392,-0.613222,-0.358885,1.625095,0.643726
3,-0.177683,-0.649530,-0.285416,-0.986173,-0.522082,-0.249346,-0.314775,-0.451759,-0.423253,-0.338585,...,-0.427851,-0.290868,-0.666745,-0.437182,-0.340548,-0.674366,-0.629239,-0.374523,-0.306995,-0.496105
4,-0.177163,-0.598919,-0.206958,0.028644,-0.522084,-0.249373,-0.313449,-0.450472,-0.358900,-0.338446,...,-0.231647,-0.267146,0.235424,0.391872,-0.244914,-0.462426,1.377628,-0.326423,-0.285082,-0.253488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0.361295,1.372549,0.722690,0.795316,-0.036477,-0.213304,-0.282888,-0.390043,-0.139379,-0.092822,...,-0.119199,-0.270142,2.348187,-0.217078,-0.258073,0.370915,-0.441337,0.225760,-0.245379,-0.183012
98,0.396554,3.741383,0.231097,1.107621,0.273080,-0.137044,-0.280623,-0.395629,0.664168,-0.182926,...,-0.414705,1.403502,-0.463925,-0.032628,-0.341533,0.199887,-0.476639,-0.134947,-0.247760,-0.370365
99,1.654014,-0.516077,0.121185,1.025451,3.148635,0.027362,-0.314703,-0.314815,-0.419479,1.387056,...,-0.362443,-0.260349,0.456551,-0.170669,0.033321,2.598063,-0.007481,-0.366481,0.334043,0.345386
100,-0.176968,0.465574,-0.284714,-0.420894,-0.505216,-0.249398,2.897313,-0.244529,0.550565,-0.338358,...,-0.397440,-0.285465,-0.248693,-0.405329,-0.091101,-0.012827,-0.624372,-0.373649,-0.308980,-0.464652


In [8]:
col_lab = list(range(0,1380))
X = X.set_axis(col_lab, axis="columns")

In [9]:
X

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,-0.177281,3.297896,-0.245501,0.650651,5.995141,-0.249386,-0.306055,-0.413578,1.077594,-0.334877,...,-0.298899,-0.287583,-0.148622,-0.234587,-0.290536,-0.640381,-0.067265,-0.376702,-0.296578,-0.428227
1,-0.171037,-0.342376,-0.260220,0.278552,-0.522190,-0.242656,-0.314110,-0.439076,-0.423111,-0.283096,...,-0.416881,-0.295374,-0.593796,-0.209265,-0.034916,-0.381576,-0.552161,-0.366302,-0.277084,-0.488131
2,-0.177402,-0.587210,0.076961,-0.745354,-0.522509,-0.187210,-0.314593,-0.091367,-0.422259,-0.326698,...,-0.409615,0.252402,-0.544256,0.474081,0.388900,0.740392,-0.613222,-0.358885,1.625095,0.643726
3,-0.177683,-0.649530,-0.285416,-0.986173,-0.522082,-0.249346,-0.314775,-0.451759,-0.423253,-0.338585,...,-0.427851,-0.290868,-0.666745,-0.437182,-0.340548,-0.674366,-0.629239,-0.374523,-0.306995,-0.496105
4,-0.177163,-0.598919,-0.206958,0.028644,-0.522084,-0.249373,-0.313449,-0.450472,-0.358900,-0.338446,...,-0.231647,-0.267146,0.235424,0.391872,-0.244914,-0.462426,1.377628,-0.326423,-0.285082,-0.253488
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,0.361295,1.372549,0.722690,0.795316,-0.036477,-0.213304,-0.282888,-0.390043,-0.139379,-0.092822,...,-0.119199,-0.270142,2.348187,-0.217078,-0.258073,0.370915,-0.441337,0.225760,-0.245379,-0.183012
98,0.396554,3.741383,0.231097,1.107621,0.273080,-0.137044,-0.280623,-0.395629,0.664168,-0.182926,...,-0.414705,1.403502,-0.463925,-0.032628,-0.341533,0.199887,-0.476639,-0.134947,-0.247760,-0.370365
99,1.654014,-0.516077,0.121185,1.025451,3.148635,0.027362,-0.314703,-0.314815,-0.419479,1.387056,...,-0.362443,-0.260349,0.456551,-0.170669,0.033321,2.598063,-0.007481,-0.366481,0.334043,0.345386
100,-0.176968,0.465574,-0.284714,-0.420894,-0.505216,-0.249398,2.897313,-0.244529,0.550565,-0.338358,...,-0.397440,-0.285465,-0.248693,-0.405329,-0.091101,-0.012827,-0.624372,-0.373649,-0.308980,-0.464652


In [18]:
import pandas as pd

snp_prvcs = pd.read_csv(path_prefix + "DNABERT_2/Datasets/PRVCS/test_dataset/Rare_Patho_SNV_dataset_XY-named.txt", sep='\t')


In [19]:
snp_prvcs

,Name,Chromosome,Position,Ref,Alt,label
0,RARPAT0001,1,22155841,T,C,0
1,RARPAT0002,1,22155853,C,T,0
2,RARPAT0003,1,22166108,C,T,0
3,RARPAT0004,1,22176438,C,T,0
4,RARPAT0005,1,55527240,C,G,0
...,...,...,...,...,...,...
97,RARPAT0098,X,38211835,C,T,1
98,RARPAT0099,X,38211844,C,A,1
99,RARPAT0100,X,48683304,A,G,1
100,RARPAT0101,X,62863977,A,C,0


In [21]:
y = snp_prvcs['label'][0:102]

In [22]:
y

0      0
1      0
2      0
3      0
4      0
      ..
97     1
98     1
99     1
100    0
101    1
Name: label, Length: 102, dtype: int64

In [12]:
#y[5240:5250]

5240    1
5241    1
5242    1
5243    1
5244    1
5245    1
5246    1
5247    0
5248    0
5249    0
Name: label, dtype: int64

In [13]:
#snp_prvcs[5240:5250]

,Name,Chromosome,Position,Ref,Alt,label
5240,SNP05241,X,113818582,G,C,1
5241,SNP05242,X,115305126,A,C,1
5242,SNP05243,X,115590299,C,G,1
5243,SNP05244,X,128870791,C,A,1
5244,SNP05245,X,135726950,A,G,1
5245,SNP05246,X,138612197,T,C,1
5246,SNP05247,X,145078733,A,G,1
5247,SNP05248,1,5318950,A,T,0
5248,SNP05249,1,5320503,C,T,0
5249,SNP05250,1,5321508,G,C,0


In [23]:
from sklearn.model_selection import train_test_split
test_size = 0.3
seed = 7
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size, random_state=seed)

In [24]:
from sklearn.metrics import accuracy_score, f1_score, matthews_corrcoef, roc_auc_score

model1 = XGBClassifier() 
model2 = XGBClassifier(n_estimators=100, max_depth=8, learning_rate=0.1, subsample=0.5) 
train_model1 = model1.fit(X_train, y_train) 
train_model2 = model2.fit(X_train, y_train) 
pred1 = train_model1.predict(X_test) 
pred2 = train_model2.predict(X_test) 
print("Accuracy_model1: %.4f" % (accuracy_score(y_test, pred1))) 
print("Accuracy_model2: %.4f" % (accuracy_score(y_test, pred2))) 
print("F1_model1: %.4f" % (f1_score(y_test, pred1)))
print("F1_model2: %.4f" % (f1_score(y_test, pred2)))
print("MCC_model1: %.4f" % (matthews_corrcoef(y_test, pred1)))
print("MCC_model2: %.4f" % (matthews_corrcoef(y_test, pred2)))
print("ROC_AUC_model1: %.4f" % (roc_auc_score(y_test, pred1))) # don't know about call: y_score vs. pred
print("ROC_AUC_model2: %.4f" % (roc_auc_score(y_test, pred2))) # don't know about call: y_score vs. pred


Accuracy_model1: 0.4839
Accuracy_model2: 0.3871
F1_model1: 0.3846
F1_model2: 0.1739
MCC_model1: 0.0092
MCC_model2: -0.2117
ROC_AUC_model1: 0.5042
ROC_AUC_model2: 0.4160


In [26]:
pred2[0:102].sum()

6

In [27]:
model3 = XGBClassifier(
 learning_rate =0.1,
 n_estimators=1000,
 max_depth=5,
 min_child_weight=1,
 gamma=0,
 subsample=0.8,
 colsample_bytree=0.8,
 objective= 'binary:logistic',
 nthread=4,
 scale_pos_weight=1,
 seed=7)

train_model3 = model3.fit(X_train, y_train)
pred3 = train_model3.predict(X_test)
print("Accuracy_model3: %.4f" % (accuracy_score(y_test, pred3)))
print("F1_model3: %.4f" % (f1_score(y_test, pred3)))
print("MCC_model3: %.4f" % (matthews_corrcoef(y_test, pred3)))
print("ROC_AUC_model3: %.4f" % (roc_auc_score(y_test, pred3))) # don't know about call: y_score vs. pred

Accuracy_model3: 0.4516
F1_model3: 0.2609
MCC_model3: -0.0476
ROC_AUC_model3: 0.4811


In [28]:
from sklearn.model_selection import GridSearchCV

param_test = {
 'max_depth':[4],
 'min_child_weight':[2],
 'learning_rate':[0.1],
 'n_estimators':[50],
 'objective':['binary:logistic']
 #'objective':['binary:logistic','binary:hinge','binary:logitraw']
}
    
gsearch = GridSearchCV(estimator=XGBClassifier(gamma=0, 
                                               subsample=0.8, 
                                               colsample_bytree=0.8, 
                                               nthread=4, 
                                               scale_pos_weight=1, 
                                               seed=7), 
                       param_grid=param_test, 
                       scoring='roc_auc',
                       n_jobs=4, 
                       verbose=4,
                       return_train_score=True,
                       cv=5,
                       refit=True)

train_model4 = gsearch.fit(X_train, y_train)
pred4 = train_model4.predict(X_test)
print("Accuracy_model4: %.4f" % (accuracy_score(y_test, pred4)))  
print("F1_model4: %.4f" % (f1_score(y_test, pred4)))  
print("MCC_model4: %.4f" % (matthews_corrcoef(y_test, pred4)))  
print("ROC_AUC_model4: %.4f" % (roc_auc_score(y_test, pred4)))  

Fitting 5 folds for each of 1 candidates, totalling 5 fits
Accuracy_model4: 0.4839
F1_model4: 0.3846
MCC_model4: 0.0092
ROC_AUC_model4: 0.5042


In [29]:
len(pred4)

31

In [30]:
gsearch.cv_results_['params'][gsearch.best_index_]

{'learning_rate': 0.1,
 'max_depth': 4,
 'min_child_weight': 2,
 'n_estimators': 50,
 'objective': 'binary:logistic'}

In [31]:
model_deepsea1 = XGBClassifier(
 reg_alpha = 20,
 reg_lambda = 2000,
 eta = 0.1,
 n_estimators=10,
 objective= 'binary:logistic',
 seed=7)

train_model_deepsea1 = model_deepsea1.fit(X_train, y_train)
pred_deepsea1 = train_model_deepsea1.predict(X_test)
print("Accuracy_model_deepsea1: %.4f" % (accuracy_score(y_test, pred_deepsea1)))
print("F1_model_deepsea1: %.4f" % (f1_score(y_test, pred_deepsea1)))
print("MCC_model_deepsea1: %.4f" % (matthews_corrcoef(y_test, pred_deepsea1)))
print("ROC_AUC_model_deepsea1: %.4f" % (roc_auc_score(y_test, pred_deepsea1))) # don't know about call: y_score vs. pred

Accuracy_model_deepsea1: 0.4516
F1_model_deepsea1: 0.0000
MCC_model_deepsea1: 0.0000
ROC_AUC_model_deepsea1: 0.5000


/cm/shared/apps/spack/0.17.3/cpu/b/opt/spack/linux-rocky8-zen/gcc-8.5.0/anaconda3-2021.05-q4munrgvh7qp4o7r3nzcdkbuph4z7375/lib/python3.8/site-packages/sklearn/metrics/_classification.py:870: RuntimeWarning: invalid value encountered in scalar divide
  mcc = cov_ytyp / np.sqrt(cov_ytyt * cov_ypyp)


In [32]:
model_deepsea2 = XGBClassifier(
 reg_alpha = 0,
 reg_lambda = 10,
 eta = 0.1,
 n_estimators=100,
 objective= 'binary:logistic',
 seed=7)

train_model_deepsea2 = model_deepsea2.fit(X_train, y_train)
pred_deepsea2 = train_model_deepsea2.predict(X_test)
print("Accuracy_model_deepsea1: %.4f" % (accuracy_score(y_test, pred_deepsea2)))
print("F1_model_deepsea2: %.4f" % (f1_score(y_test, pred_deepsea2)))
print("MCC_model_deepsea2: %.4f" % (matthews_corrcoef(y_test, pred_deepsea2)))
print("ROC_AUC_model_deepsea2: %.4f" % (roc_auc_score(y_test, pred_deepsea2))) # don't know about call: y_score vs. pred

Accuracy_model_deepsea1: 0.5806
F1_model_deepsea2: 0.4800
MCC_model_deepsea2: 0.2389
ROC_AUC_model_deepsea2: 0.6050


In [33]:
model_regbase = XGBClassifier(
    colsample_bylevel = 0.6,
    colsample_bytree = 1.0,
    gamma = 5.0,
    learning_rate = 0.1,
    max_depth = 4,
    min_child_weight = 4,
    n_estimators = 300,
    reg_alpha = 5.0,
    reg_lambda = 0.01,
    scale_pos_weight = 10.658093373,
    subsample = 1.0)

train_model_regbase = model_regbase.fit(X_train, y_train)
pred_regbase = train_model_regbase.predict(X_test)
print("Accuracy_model_regbase: %.4f" % (accuracy_score(y_test, pred_regbase)))
print("F1_model_regbase: %.4f" % (f1_score(y_test, pred_deepsea2)))
print("MCC_model_regbase: %.4f" % (matthews_corrcoef(y_test, pred_regbase)))
print("ROC_AUC_model_regbase: %.4f" % (roc_auc_score(y_test, pred_regbase)))

Accuracy_model_regbase: 0.5484
F1_model_regbase: 0.4800
MCC_model_regbase: 0.0374
ROC_AUC_model_regbase: 0.5126


In [25]:
# Save best model

In [26]:
from joblib import dump, load

best_estimator = train_model4
dump(best_estimator, path_prefix + "DNABERT_2/Output_Models/" + "model_xpg_boost_PRVCS_REG.joblib")

['/expanse/lustre/projects/nji102/sgriesmer/DNABERT_2/Output_Models/model_xpg_boost_PRVCS_REG.joblib']

In [27]:
test_estimator = load('/expanse/lustre/projects/nji102/sgriesmer/DNABERT_2/Output_Models/model_xpg_boost_PRVCS_REG.joblib')

In [28]:
test_pred = test_estimator.predict(X_test)

In [29]:
test_pred.sum()/3148

0.4729987293519695

In [30]:
# Load SNP data

In [31]:
SNPlt01_abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/lt05_igap_ran_0_10000-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')

In [32]:
# Set model names to numbers

In [33]:
SNPlt01_abs_diff_plus_log_odds_scores_norm.set_axis(col_lab, axis="columns", inplace=True)

In [34]:
SNPlt01_abs_diff_plus_log_odds_scores_norm

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,-0.134340,-0.176669,-0.167205,1.136512,0.598432,-0.162685,-0.172438,0.035790,-0.143567,-0.154109,...,-0.316542,-0.200087,-0.322255,-0.241135,-0.097094,-0.150122,-0.420258,-0.225923,-0.196953,-0.289094
1,-0.134376,-0.366801,-0.167250,-0.652682,-0.361398,-0.162674,-0.203901,-0.443247,-0.298716,-0.154114,...,0.419565,-0.195666,-0.377331,-0.174015,-0.222661,-0.306294,-0.472069,-0.218917,-0.197477,-0.280357
2,-0.023767,-0.197831,-0.167147,0.070671,-0.200981,0.235543,-0.203821,-0.207416,-0.223960,-0.147906,...,2.071424,-0.186225,-0.038674,0.445475,0.074265,0.001689,-0.210508,-0.193521,-0.186807,0.506437
3,-0.134375,-0.437801,-0.167141,-0.155040,-0.285044,-0.162685,-0.203818,-0.269907,-0.297292,-0.154112,...,-0.319017,-0.198726,-0.384140,-0.275544,-0.037049,-0.276440,-0.447298,-0.224284,-0.196686,-0.247948
4,-0.134354,-0.498539,-0.167261,-0.653013,-0.356434,-0.162685,-0.203906,-0.443459,-0.298768,-0.154109,...,-0.278643,-0.200142,-0.378112,-0.200160,-0.197726,-0.271060,-0.457370,-0.226998,-0.196294,-0.271508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.134333,-0.102992,-0.167079,0.438134,-0.358858,-0.162685,-0.203737,5.375807,-0.297380,-0.154109,...,-0.316421,-0.195129,-0.325719,-0.169471,-0.159812,0.021548,-0.315698,-0.225919,-0.197648,-0.276390
9996,-0.130458,-0.029596,-0.127534,1.780980,-0.002129,-0.162602,-0.203783,-0.264749,0.331581,-0.145322,...,0.118872,-0.152362,-0.317824,0.050219,0.247068,-0.077249,-0.193526,-0.225535,-0.169228,0.006952
9997,-0.134323,-0.276844,-0.167105,-0.526303,-0.351880,-0.162685,-0.198805,-0.370694,-0.266654,-0.154111,...,-0.309555,-0.190732,-0.338031,-0.288699,-0.218013,-0.282265,-0.440474,-0.219109,-0.186410,-0.270539
9998,-0.134193,0.185418,-0.165822,-0.172234,-0.355526,-0.162660,-0.203906,-0.435158,-0.297253,-0.153950,...,-0.224890,-0.168852,-0.143209,-0.260668,-0.245686,-0.070232,-0.192810,-0.107638,-0.195652,0.022638


In [35]:
SNPlt01_pred = test_estimator.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)

In [36]:
SNPlt01_pred.sum()

9999

In [37]:
SNPgt5_abs_diff_plus_log_odds_scores_norm = pd.read_csv(path_prefix + "DNABERT_2/output/gt5_igap_ran_0_10000-abs_diff_plus_log_odds_scores_norm_results_by_variant.csv", sep=',')

In [38]:
SNPgt5_abs_diff_plus_log_odds_scores_norm.set_axis(col_lab, axis="columns", inplace=True)

In [39]:
SNPgt5_abs_diff_plus_log_odds_scores_norm

,0,1,2,3,4,5,6,7,8,9,...,1370,1371,1372,1373,1374,1375,1376,1377,1378,1379
0,-0.134340,-0.176669,-0.167205,1.136512,0.598432,-0.162685,-0.172438,0.035790,-0.143567,-0.154109,...,-0.316542,-0.200087,-0.322255,-0.241135,-0.097094,-0.150122,-0.420258,-0.225923,-0.196953,-0.289094
1,-0.134376,-0.366801,-0.167250,-0.652682,-0.361398,-0.162674,-0.203901,-0.443247,-0.298716,-0.154114,...,0.419565,-0.195666,-0.377331,-0.174015,-0.222661,-0.306294,-0.472069,-0.218917,-0.197477,-0.280357
2,-0.023767,-0.197831,-0.167147,0.070671,-0.200981,0.235543,-0.203821,-0.207416,-0.223960,-0.147906,...,2.071424,-0.186225,-0.038674,0.445475,0.074265,0.001689,-0.210508,-0.193521,-0.186807,0.506437
3,-0.134375,-0.437801,-0.167141,-0.155040,-0.285044,-0.162685,-0.203818,-0.269907,-0.297292,-0.154112,...,-0.319017,-0.198726,-0.384140,-0.275544,-0.037049,-0.276440,-0.447298,-0.224284,-0.196686,-0.247948
4,-0.134354,-0.498539,-0.167261,-0.653013,-0.356434,-0.162685,-0.203906,-0.443459,-0.298768,-0.154109,...,-0.278643,-0.200142,-0.378112,-0.200160,-0.197726,-0.271060,-0.457370,-0.226998,-0.196294,-0.271508
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,-0.134333,-0.102992,-0.167079,0.438134,-0.358858,-0.162685,-0.203737,5.375807,-0.297380,-0.154109,...,-0.316421,-0.195129,-0.325719,-0.169471,-0.159812,0.021548,-0.315698,-0.225919,-0.197648,-0.276390
9996,-0.130458,-0.029596,-0.127534,1.780980,-0.002129,-0.162602,-0.203783,-0.264749,0.331581,-0.145322,...,0.118872,-0.152362,-0.317824,0.050219,0.247068,-0.077249,-0.193526,-0.225535,-0.169228,0.006952
9997,-0.134323,-0.276844,-0.167105,-0.526303,-0.351880,-0.162685,-0.198805,-0.370694,-0.266654,-0.154111,...,-0.309555,-0.190732,-0.338031,-0.288699,-0.218013,-0.282265,-0.440474,-0.219109,-0.186410,-0.270539
9998,-0.134193,0.185418,-0.165822,-0.172234,-0.355526,-0.162660,-0.203906,-0.435158,-0.297253,-0.153950,...,-0.224890,-0.168852,-0.143209,-0.260668,-0.245686,-0.070232,-0.192810,-0.107638,-0.195652,0.022638


In [40]:
SNPgt5_pred = test_estimator.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)

In [41]:
SNPgt5_pred.sum()

9999

In [42]:
# Try other models

In [43]:
pred1_lt01 = train_model1.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm) 
pred2_lt01 = train_model2.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm) 
pred3_lt01 = train_model3.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)
pred4_lt01 = train_model4.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)
pred_deepsea1_lt01 = train_model_deepsea1.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)
pred_deepsea2_lt01 = train_model_deepsea2.predict(SNPlt01_abs_diff_plus_log_odds_scores_norm)

In [44]:
for i in [
    pred1_lt01,
    pred2_lt01,
    pred3_lt01,
    pred4_lt01,
    pred_deepsea1_lt01,
    pred_deepsea2_lt01,
  ]:
    print(i.sum())

9925
9915
9973
9999
10000
9991


In [45]:
pred1_gt5 = train_model1.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm) 
pred2_gt5 = train_model2.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm) 
pred3_gt5 = train_model3.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)
pred4_gt5 = train_model4.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)
pred_deepsea1_gt5 = train_model_deepsea1.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)
pred_deepsea2_gt5 = train_model_deepsea2.predict(SNPgt5_abs_diff_plus_log_odds_scores_norm)

In [46]:
for i in [
    pred1_gt5,
    pred2_gt5,
    pred3_gt5,
    pred4_gt5,
    pred_deepsea1_gt5,
    pred_deepsea2_gt5,
  ]:
    print(i.sum())

9925
9915
9973
9999
10000
9991
